# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [54]:
# import numpy and pandas
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind
from scipy.stats import f_oneway
import scipy
from scipy import stats
from scipy.stats import t

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [3]:
# Your code here:
salaries = pd.read_csv("Current_Employee_Names__Salaries__and_Position_Titles.csv")

Examine the `salaries` dataset using the `head` function below.

In [71]:
# Your code here:

salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
5,"ABBASI, CHRISTOPHER",STAFF ASST TO THE ALDERMAN,CITY COUNCIL,F,Salary,NaN,50436.0,NaN
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
8,"ABBATEMARCO, JAMES J",FIRE ENGINEER-EMT,FIRE,F,Salary,NaN,103350.0,NaN
9,"ABBATE, TERRY M",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [6]:
# Your code here:
salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [10]:
# Your code here:
salaries["Salary or Hourly"].count()

33183

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [12]:
# Your code here:

salaries.pivot_table(values = ["Name"], index=["Department"], aggfunc = "count")

,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [21]:
# Your code here:
ttest_1samp(salaries["Hourly Rate"], 30, nan_policy='omit')

Ttest_1sampResult(statistic=20.619805785494147, pvalue=4.323024048614701e-92)

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [73]:
# Your code here:
new = salaries[salaries["Department"] == "POLICE"]

ttest_1samp(new["Annual Salary"], 86000 , nan_policy='omit')

#the p value is positive so the hypotesis is false.

Ttest_1sampResult(statistic=3.081997005712994, pvalue=0.00206034035510054)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [45]:
# Your code here:

table = pd.crosstab(salaries["Department"], salaries["Typical Hours"], margins = True)
table

#The department with more hours is the STREETS & SAN.

Typical Hours,10.0,20.0,35.0,40.0,All
Department,,,,,
ANIMAL CONTRL,1,18,0,0,19
AVIATION,0,18,15,1049,1082
BUDGET & MGMT,0,0,2,0,2
BUSINESS AFFAIRS,0,7,0,0,7
CITY COUNCIL,7,36,18,3,64
COMMUNITY DEVELOPMENT,0,3,1,0,4
CULTURAL AFFAIRS,0,0,7,0,7
FAMILY & SUPPORT,0,286,0,1,287
FINANCE,0,2,0,42,44


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [47]:
# Your code here:
new1 = salaries[salaries["Department"] == "STREETS & SAN"]
new1["Hourly Rate"].mean()

# If the mean value is less than 35$/hour then the workers are right.
#ttest_1samp(new1["Hourly Rate"], 35, nan_policy='omit')

33.728378088076845

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [77]:
# Your code here:

a = salaries["Hourly Rate"].dropna() 
se = stats.sem(a)
mean = np.mean(a)
ci = t.interval(0.95, 1, mean, se)
ci

(31.070210529551705, 34.50690490301086)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [76]:
# Your code here:
a = new["Annual Salary"].dropna() 
#police works 20h a week.
#A year was 52 weeks so a police works 1040 h without vacations
#according to google a police was the right of 4.62 hours of vacation every two weeks so every year was ~120h vacation
#so a police officer works aprox. 920h
a = a/920
se = stats.sem(a)
mean = np.mean(a)
ci = t.interval(0.95, 1, mean, se)
ci


(91.8272441774366, 96.1867003945851)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

